# Association Analysis

Association analysis uses machine learning to extract hidden relationships from large datasets. In this assignment you'll be implementing two of the most commonly used algorithms for association rule mining: Apriori and FP-Growth.

The dataset (`large_retail.txt`) that we are going to use for this assignment has been adapted from the [Retail Market Basket Dataset](http://fimi.ua.ac.be/data/retail.pdf). The dataset contains transaction records supplied by an anonymous Belgian retail supermarket store. Each line in the file represents a separate transaction with the item ids separated by space. The dataset has 3000 transaction records and 99 different item ids.

We also provide a smaller dataset (`small_retail.txt`) with 9 transactions and 5 different item ids along with the solutions. *You should first test your implementation on this dataset, before running it on the larger dataset.*

The assignment will be **autograded**. We will use the `diff` command in linux to compare the output files. So please **check your answers** based on the given sample output files.

Implementation Hint:

- Use the `frozenset` data structure in Python (similar to `set` in functionality) to represent the itemsets because `frozenset` is a hashable data structure. You can maintain a dictionary that maps from the itemset (a `frozenset`) to its support count.

## Part 1 - Apriori Algorithm

Apriori algorithm is a classical algorithm in data mining. It is used for mining frequent itemsets and relevant association rules. In this part, you'll be implementing this algorithm for generating the itemsets that occur more than the `min_sup` threshold. Based on these frequent itemsets you'll find association rules that have confidence above the `min_conf` threshold.

In [2]:
# Standard imports (you can add additional headers if you wish)
import numpy as np

In [3]:
# Reading the dataset from file
def load_dataset(file_name):
    with open(file_name, 'r') as f:
        content = f.readlines()
        data = [[int(x) for x in line.rstrip().split()] for line in content]
    return data

In [8]:
small_dataset = load_dataset('small_retail.txt')
small_dataset

[[1, 2, 5],
 [2, 4],
 [2, 3],
 [1, 2, 4],
 [1, 3],
 [2, 3],
 [1, 3],
 [1, 2, 3, 5],
 [1, 2, 3]]

**Q1.** Implement the function `create_1_itemsets` that takes as input the entire dataset and returns a list of all the 1-itemsets. For example, for `small_retail.txt` it should return:
~~~
[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]
 ~~~
 Please **don't hardcode** the item ids, your code should support item ids that are non-sequential.

In [4]:
def create_1_itemsets(dataset):
    c1 = []
    aux_list = []
    # your code goes here
    for transaction in dataset:
        for x in transaction:
            if x not in aux_list:
                aux_list.append(x)
    aux_list = sorted(aux_list)
    for x in aux_list:
        c1.append(frozenset([x]))
    return c1

**Q2.** Implement function `filter_candidates` that takes as input the candidate itemsets, the dataset, and the minumum support count `min_sup`, and filters out candidates that don't meet the support threshold.

**Hint:** You should also return the support count information (perhaps as a `dict`) for the itemsets. This will be useful later on for the final output.

In [5]:
def filter_candidates(candidates, dataset, min_sup):
    retlist = []
    support_data = {}
    # your code goes here
    for itemset in candidates:
        for transaction in dataset:
            if itemset.issubset(set(transaction)):
                if itemset not in support_data:
                    support_data[itemset] = 1
                else:
                    support_data[itemset] += 1
    # Adding missing candidates
    for itemset in candidates:
        if itemset not in support_data:
            support_data[itemset] = 0
    # Choosing itemsets that have min_sup
    for itemset in support_data:
        if support_data[itemset] >= min_sup:
            retlist.append(itemset)
    return retlist, support_data

In [28]:
candidates = create_1_itemsets(small_dataset)
print(candidates)
retlist, support_data = filter_candidates(candidates, small_dataset, 6)
print(retlist)
print(support_data)

[frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]
[frozenset({1}), frozenset({2}), frozenset({3})]
{frozenset({1}): 6, frozenset({2}): 7, frozenset({3}): 6, frozenset({4}): 2, frozenset({5}): 2}


**Q3.** Implement the function `generate_next_itemsets` that takes in frequent itemsets of size `k` and generates candidate itemsets of size `k + 1`.

**Hint:** Use the fact that if `[1, 2, 3, 4]` is a frequent itemset of size 4 then `[1, 2, 3]` and `[1, 2, 4]` both will be frequent itemsets of size 3. You can use this to drastically reduce the number of candidate itemsets that you need to generate. (Use the F(k-1) x F(k-1) or F(k-1) x F(1) candidate generation method).

In [50]:
freq_sets = [frozenset([1,2,3]), frozenset([1,2,4])]

x = generate_next_itemsets(freq_sets)

print(x)


[frozenset({1, 2, 3, 4})]


In [6]:
def generate_next_itemsets(freq_sets):
    retlist = []
    aux_ret = []
    aux_list = []
    
    # List of lists
    for itemset in freq_sets:
        itemset = sorted(itemset)
        aux_list.append(itemset)
    # Sorted list
    aux_list = sorted(aux_list)
    
    # Fk-1 vs Fk-1
    for lst in aux_list:
        for vs in aux_list:
            if (lst[:-1] == vs[:-1]) and (lst[-1]!=vs[-1]):
                aux_ret.append(lst+[vs[-1]])
    
    # Back to frozenset
    for lst in aux_ret:
        if frozenset(lst) not in retlist:
            retlist.append(frozenset(lst))
    
    return retlist

**Q4.** Implement the function `apriori_freq_temsets` that takes the entire dataset as the input and returns the frequent itemsets that have support count more than `min_sup`.

In [7]:
def apriori_freq_temsets(dataset, minsup):
    apriori_itemsets = []
     # your code goes here
    candidate_1_itemset = create_1_itemsets(dataset)
    retlst, support_data = filter_candidates(candidate_1_itemset, dataset, minsup)
    while len(retlst) > 0:
        apriori_itemsets = apriori_itemsets + retlst
        candidates = generate_next_itemsets(retlst)
        retlst, support_data = filter_candidates(candidates, dataset, minsup)
    return apriori_itemsets

**Q5.** Display the frequent item sets in the form of a table along with their `support_fraction` for the `large_retail.txt` dataset with min support count 300.

Sample Table Format (tab separated table)

~~~
Sup     Freq Itemset
0.67	[1]
0.44	[1, 2]
(and so on)
...
...
~~~

`support_fraction(itemset) = support_count(itemset) / num_total_transactions`.

Note that the `support_fraction` should be rounded to the nearest 2 decimal places (use `round(sup, 2)`). Also `support_fraction` and the itemset should be separated by a tab (`'\t'`). The itemsets should also be in a sorted order where smaller itemsets should come before larger itemsets and itemsets of the same size should be sorted amongst themselves.

For eg. 
~~~~
[1, 2] should come before [1, 2, 3]
[1, 2, 3] should come before [1, 2, 4]
[1, 2, 3] should come before [1, 4, 5]
[1, 2, 3] should come before [2, 3, 4]
~~~~

Note that **this order is very important** because your output will be checked using the `diff` command. The output also **shouldn't contain any duplicates**. The sample output for the `small_retail.txt` dataset with min support count as 2 is:

~~~~
Sup     Freq Itemset
0.67	[1]
0.78	[2]
0.67	[3]
0.22	[4]
0.22	[5]
0.44	[1, 2]
0.44	[1, 3]
0.22	[1, 5]
0.44	[2, 3]
0.22	[2, 4]
0.22	[2, 5]
0.22	[1, 2, 3]
0.22	[1, 2, 5]
~~~~

**Store** this output for the `large_retail.txt` dataset in the file `apriori_itemsets.txt`. The sample output file for the `small_retail.txt` dataset has been provided to you as `small_apriori_itemsets.txt` for your convenience.

In [95]:
large_dataset = load_dataset('large_retail.txt')
results = apriori_freq_temsets(large_dataset, 300)
retlst, support_data = filter_candidates(results, large_dataset, 300)
f = open("apriori_itemsets.txt","w+")
print("Sup\tFreq Itemset\n")
f.write("Sup\tFreq Itemset\n")
count = 0
for itemset, sup in support_data.items():
    print(str(round(sup/len(large_dataset),2)) + '\t' + str(sorted(list(itemset)))+'\n')
    if count < len(support_data)-1:
        f.write(str(round(sup/len(large_dataset),2)) + '\t' + str(sorted(list(itemset)))+'\n')
    else:
        f.write(str(round(sup/len(large_dataset),2)) + '\t' + str(sorted(list(itemset))))
    count = count + 1
f.close()

Sup	Freq Itemset

0.1	[31]

0.22	[32]

0.11	[36]

0.26	[38]

0.6	[39]

0.31	[41]

0.47	[48]

0.11	[60]

0.11	[65]

0.11	[89]

0.14	[32, 39]

0.12	[32, 48]

0.17	[38, 39]

0.11	[38, 41]

0.13	[38, 48]

0.23	[39, 41]

0.33	[39, 48]

0.18	[41, 48]

0.14	[39, 41, 48]



**Q6.** Find the closed frequent item sets along with their `support_fraction`. Store the results for the `large_retail.txt` dataset in the file `apriori_closed_itemsets.txt`, in the same format as specified in Q5.

In [108]:
# your code goes here
large_dataset = load_dataset('large_retail.txt')
results = apriori_freq_temsets(large_dataset, 300)
retlst, support_data = filter_candidates(results, large_dataset,300)
closed_itemsets = {}
for itemset, sup in support_data.items():
    for item,sup_item in closed_itemsets.copy().items():
        if itemset.issuperset(item) and sup>=sup_item:
            del closed_itemsets[item]
    closed_itemsets[itemset]=sup
f = open("apriori_closed_itemsets.txt","w+")
print("Sup\tFreq Itemset\n")
f.write("Sup\tFreq Itemset\n")
count = 0
for itemset, sup in closed_itemsets.items():
    print(str(round(sup/len(large_dataset),2)) + '\t' + str(sorted(list(itemset)))+'\n')
    if count < len(closed_itemsets)-1:
        f.write(str(round(sup/len(large_dataset),2)) + '\t' + str(sorted(list(itemset)))+'\n')
    else:
        f.write(str(round(sup/len(large_dataset),2)) + '\t' + str(sorted(list(itemset))))
    count = count + 1
f.close()

Sup	Freq Itemset

0.1	[31]

0.22	[32]

0.11	[36]

0.26	[38]

0.6	[39]

0.31	[41]

0.47	[48]

0.11	[60]

0.11	[65]

0.11	[89]

0.14	[32, 39]

0.12	[32, 48]

0.17	[38, 39]

0.11	[38, 41]

0.13	[38, 48]

0.23	[39, 41]

0.33	[39, 48]

0.18	[41, 48]

0.14	[39, 41, 48]



## Part 2 - Frequent Pattern Tree

The FP-Growth Algorithm, proposed by [Han](https://www.cs.sfu.ca/~jpei/publications/sigmod00.pdf), is an efficient and scalable method for mining the complete set of frequent patterns by pattern fragment growth, using an extended prefix-tree structure for storing compressed and crucial information about frequent patterns named frequent-pattern tree (FP-tree).

Use the following FPTreeNode class to build your FP Tree. You may add methods to it if you wish.

In [3]:
# variables:
# self.item_id: item id of the item
# self.item_count: item count for this node
# self.node_link: next pointer for the linked list that connects
#                 nodes of the same item_id (required for the FP-growth algorithm)
# self.parent: pointer to the parent node (should be None for the root)
# self.children: dictionary for the children (maps from item_id to the FPTreeNode object)
# NOTE: root node should have item_id as -1

class FPTreeNode:
    
    def __init__(self, uid, num):
        self.item_id = uid
        self.item_count = num
        self.node_link = None
        self.parent = None
        self.children = {}
        
    def displayTree(self, tab=1):
        if self.item_id == -1:
            print ('  '*tab, 'root')
        else:
            print ('  '*tab, 'item_id:', self.item_id, 'item_count:', self.item_count)
        for key in sorted(self.children.keys()):
            self.children[key].displayTree(tab + 1)
            
    # helper function for saveToFile
    def saveToFile_helper(self, fp, tab=1):
        if self.item_id == -1:
            print ('  '*tab, 'root', file=fp)
        else:
            print ('  '*tab, 'item_id:', self.item_id, 'item_count:', self.item_count, file=fp)
        for key in sorted(self.children.keys()):
            self.children[key].saveToFile_helper(fp, tab + 1)
    
    # call this to save to file
    def saveToFile(self, filename):
        with open(filename, 'w') as fp:
            self.saveToFile_helper(fp)

**Q7.** Write the function `order_transactions` that takes in the dataset and reorders the items in each transaction based on their frequency (or support counts) and also removes any items that do not meet minsup. This function should return the reordered transactions as well as a dictionary containing the support counts of the frequent 1-itmesets.

Note that **item_ids that have equal frequency should be ordered based on the item_id value**.

For example,
~~~~
For the small_retail.txt dataset:
{item_id: frequency} -> {1: 6, 2: 7, 3: 6, 4: 2, 5: 2}
The transaction [1, 2, 3, 5] should be reordered as [2, 1, 3, 5]
Notice the ordering of 1 and 3 (both have 6 occurences in the dataset)

If minsup=3, [1, 2, 3, 5] should be reordered as [2, 1, 3] because 5 does not meet minsup.
~~~~

Note that the **relative order between transactions should not be changed**, they remain in the same order as they appear in the original dataset. For the `small_retail.txt` the transactions would be inserted in the FP Tree in this order:
~~~~
[2, 1, 5]
[2, 4]
[2, 3]
[2, 1, 4]
[1, 3]
[2, 3]
[1, 3]
[2, 1, 3, 5]
[2, 1, 3]
~~~~

In [4]:
def sortByMinsup(item,freq_cnt):
    return freq_cnt[item]

In [27]:
def order_transactions(dataset, minsup):
    freq_cnt = {}
    return_freq = {}
    new_dataset = []
    aux_transaction = []
    # your code goes here
    for transaction in dataset:
        for item in transaction:
            if item not in freq_cnt:
                freq_cnt[item] = 1
            else:
                freq_cnt[item] += 1
    for transaction in dataset:
        for item in transaction:
            if freq_cnt[item] >= minsup:
                aux_transaction.append(item)
                aux_transaction.sort(key=lambda x: sortByMinsup(x, freq_cnt),reverse=True)
        new_dataset.append(aux_transaction)
        aux_transaction = []
    #Order freq_cont
    for item in sorted(freq_cnt.keys()):
        if freq_cnt[item] >= minsup:
            return_freq[item] = freq_cnt[item]
    return new_dataset, return_freq

In [92]:
new_dataset, freq_cnt = order_transactions(small_dataset, 1)
print(new_dataset)
print(freq_cnt)

print(items_des)
root, lst = build_fp_tree(new_dataset)
root.displayTree()
print(is_single_path(root))
print(*lst)
x = build_cond_pat_base(lst[5][0])
print(x)
root, lst = build_fp_tree([[2,1,5]])
root.displayTree()
print(is_single_path(root))

[[2, 1, 5], [2, 4], [2, 3], [2, 1, 4], [1, 3], [2, 3], [1, 3], [2, 1, 3, 5], [2, 1, 3]]
{1: 6, 2: 7, 3: 6, 4: 2, 5: 2}
[4, 5, 1, 3, 2]
   root
     item_id: 1 item_count: 2
       item_id: 3 item_count: 2
     item_id: 2 item_count: 7
       item_id: 1 item_count: 4
         item_id: 3 item_count: 2
           item_id: 5 item_count: 1
         item_id: 4 item_count: 1
         item_id: 5 item_count: 1
       item_id: 3 item_count: 2
       item_id: 4 item_count: 1
False
2 1 5 4 3
{(2, 1): 1, (2, 1, 3): 1}
   root
     item_id: 2 item_count: 1
       item_id: 1 item_count: 1
         item_id: 5 item_count: 1
True


**Q8.** Implement the function `build_FP_tree` which takes the ordered itemsets from Q7 and inserts them into the FP Tree. Build the FP Tree for the `large_retail.txt` dataset and write the resulting tree out to `large_fp_tree.txt`.

The tree output for `small_retail.txt` dataset is given as follows:
~~~~
   root
     item_id: 1 item_count: 2
       item_id: 3 item_count: 2
     item_id: 2 item_count: 7
       item_id: 1 item_count: 4
         item_id: 3 item_count: 2
           item_id: 5 item_count: 1
         item_id: 4 item_count: 1
         item_id: 5 item_count: 1
       item_id: 3 item_count: 2
       item_id: 4 item_count: 1
~~~~
This output has been provided to you as `small_fp_tree.txt` for your convenience. You can use the `diff` command in Linux to check your output with the provided output.




In [14]:
def build_fp_tree (ordered_transactions):
    """
    Builds an FP tree; importantly, links nodes with the same item_id and returns the first 
    node for each item_id in the first_linked_nodes dictionary
    
    parameters:
        ordered_transactions: the transactions with items sorted in descending rate of occurence

    returns:
        root (FPTreeNode): the root node of the FP tree
        first_linked_nodes (dict of int: FPTreeNode): a map from an item_id to first node of that
                                                        item_id; following each node's node_link should 
                                                        yield every node with that item_id
    """        
    root = FPTreeNode(-1, 0)
    first_linked_nodes = {}
    count = 0
    first_item = None
    previous_node = None
    
    # Your code here
    for transaction in ordered_transactions:
        for item in transaction:
            if first_item==None:
                if item not in root.children:
                    root.children[item] = FPTreeNode(item, 1)
                    root.children[item].parent = -1
                    if item not in first_linked_nodes:
                        first_linked_nodes[item] = [root.children[item]]
                    else:
                        first_linked_nodes[item][-1].node_link = root.children[item]
                        first_linked_nodes[item].append(root.children[item])
                else:
                    root.children[item].item_count += 1
                first_item = item
                previous_node = root.children[first_item]
            else:
                x = 1
                aux_dict = root.children[first_item].children
                while x < count:
                    aux_dict = aux_dict[transaction[x]].children
                    x = x + 1
                if item not in aux_dict:
                    aux_dict[item] = FPTreeNode(item,1)
                    aux_dict[item].node_link = root.children[first_item]
                    aux_dict[item].parent = previous_node
                    if item not in first_linked_nodes:
                        first_linked_nodes[item] = [aux_dict[item]]
                    else:
                        first_linked_nodes[item][-1].node_link = aux_dict[item]
                        first_linked_nodes[item].append(aux_dict[item])
                else:
                    aux_dict[item].item_count += 1
                previous_node = aux_dict[item]
            count = count + 1
        first_item = None
        count = 0
    
    return root, first_linked_nodes


In [93]:
large_dataset = load_dataset('large_retail.txt')
new_dataset, freq_cnt = order_transactions(large_dataset, 300)
root, lst = build_fp_tree(new_dataset)
root.saveToFile("large_fp_tree.txt")

**Q9.** Implement `build_cond_pat_base` which take in an FPTreeNode (representing an item) and builds its conditional pattern base. This essentially enumerates all paths in the tree that contain item's item_id.

The item (FPTreeNode) that gets passed in to this function will be an FPTreeNode in your header table. This function will first find the path leading to this node. Then it will move to the next FPTreeNode with this same item_id by using `item.node_link` and find the path leading to that node. It will iterate over all items with this item_id by using `item.node_link`.

NOTE: The conditional pattern base does not contain item itself. 

Return the conditional pattern base as a dictionary of tuples to ints representing the path to the item an how many of that item this path gets you to. For example, in the conditional pattern base for item_id 5 in `small_retail.txt` would be: `{(2, 1): 1, (2, 1, 3): 1}`

In [174]:
def build_cond_pat_base(item):
    """
    Builds a conditional pattern base for an item, starting with the first FPTreeNode of this item 
    and traversing the node_links to reach all other FPTreeNodes of this item.
    
    parameters:
        item (FPTreeNode): the node we are building a conditional pattern base for
    returns:
        cond_pat_base (dict of tuple of int: int): conditional pattern base; the patterns with which item's 
                                                   item_id is found in transactions with and their associated
                                                   counts
    """
    cond_pat_base = {}
    # your code goes here
    list_parent = []
    aux_item = item
    while aux_item != None:
        aux_parent = aux_item.parent
        while aux_parent != -1:
            list_parent.append(aux_parent.item_id)
            aux_parent = aux_parent.parent
        if list_parent != []:
            cond_pat_base[tuple(list_parent[::-1])] = aux_item.item_count
        list_parent = []
        aux_item = aux_item.node_link
    
    return cond_pat_base

**Q10.** Implement the function `is_single_path` that takes in an FP-Tree root node and determines whether or not it is a single-path tree (no branches). Return True for a single-path tree, and False otherwise.

In [16]:
def is_single_path(fp_tree):  
    # your code goes here
    aux_root = fp_tree
    while len(aux_root.children) != 0:
        if len(aux_root.children) > 1:
            return False
        only_children = list(aux_root.children.keys())[0]  
        aux_root = aux_root.children[only_children]
    return True

**Q11.** Now, implement the FP Growth algorithm to mine the FP-Tree for frequent patterns. 

Starting from the least frequent 1-itemsets:
    1. build the conditional pattern base
    2. build a new FP tree from the conditional pattern base
You then need to handle the following cases:
    1. the newly constructed FP tree has no children
    2. the newly constructed FP tree is a single path tree
    3. the newly constructed FP tree is not a single path tree
    
    If the new FP tree is not a single path tree, you will need to recurse.
    If the new FP tree is a single path tree, you will need to enumerate the subsets of the path.

You may add addition helper functions as you see fit. But remember you are only allowed to mine the FP tree that you generate in Q8. You *cannot use the dataset* other than in the first line of code provided here.

**Store** the output of mining the `large_retail.txt` dataset with a minsup=300 in the file `fp_growth_itemsets.txt`.

Hint: You can check your answer by comparing it to the output of your Apriori algorithm.

In [255]:
def make_dataset(cond_path):
    dataset = []
    transaction = []
    for path, count in cond_path.items():
        for x in range(0,count):
            for element in path:
                transaction.append(element)
            dataset.append(transaction)
            transaction = []
    return dataset

In [266]:
def recursion(dataset,minsup,final_freq_itemsets,y):
    ordered_transactions, freq_items = order_transactions(dataset, minsup)
    fp_tree, link_table = build_fp_tree(ordered_transactions)
 
    if len(fp_tree.children) == 0:
        if fp_tree.item_id != -1:
            if y not in final_freq_itemsets:
                final_freq_itemsets[y] = [([fp_tree.item_id], fp_tree.item_count)]
            else:
                final_freq_itemsets[y] += [(fp_tree.item_id, fp_tree.item_count)]
        return
    if is_single_path(fp_tree) == True:
        aux_root = fp_tree
        lst_path = []
        while len(aux_root.children) != 0:
            only_children = list(aux_root.children.keys())[0]
            lst_path.append(only_children)
            aux_root = aux_root.children[only_children]
        if y not in final_freq_itemsets:
            final_freq_itemsets[y] = [(lst_path, aux_root.item_count)]
        else:
            final_freq_itemsets[y] += [(lst_path, aux_root.item_count)]
    else:
        items_des = []
        for val in sorted(freq_items,key=freq_items.get):
            items_des.append(val)
        for x in items_des:
            first_node = link_table[x][0]
            cond_pat = build_cond_pat_base(first_node.node_link)
            datas = make_dataset(cond_pat)
            recursion(datas,minsup,final_freq_itemsets,y)

In [264]:
def fp_growth(dataset, minsup):
    ordered_transactions, freq_items = order_transactions(dataset, minsup)
    fp_tree, link_table = build_fp_tree(ordered_transactions)
    final_freq_itemsets = {}
    
    if len(fp_tree.children) != 0:
        items_des = []
        for val in sorted(freq_items,key=freq_items.get):
            items_des.append(val)
        for x in items_des:
            first_node = link_table[x][0]
            cond_pat = build_cond_pat_base(first_node)
            datas = make_dataset(cond_pat)
            recursion(datas,minsup,final_freq_itemsets,x)
    return final_freq_itemsets

In [280]:
large_dataset = load_dataset('large_retail.txt')
final_freq_itemsets = fp_growth(large_dataset, 300)
f = open("fp_growth_itemsets.txt","w+")
f.write("Sup\tFreq Itemset\n")
for val in final_freq_itemsets.keys():
    tup = final_freq_itemsets[val]
    lst = tup[0][0]
    count = tup[0][1]
    #print(str(round(count/len(small_dataset),2)) + '\t' + str(sorted(list(lst)+[val]))+'\n')
    f.write(str(round(count/len(large_dataset),2)) + '\t' + str(sorted(list(lst)+[val]))+'\n')
f.close()

## Part 3 - Extra Credit (+5 points)

**Q12.** Generate the rules having confidence above `min_conf = 0.5` using the frequent itemsets generated in Q5. Display the rules in the form of a table.

Sample table format (tab separated table)

~~~
Sup     Conf    Rule
0.44	0.67	[1] -> [2]
0.22	1.0	 [5] -> [1, 2]
0.22	1.0	 [2, 5] -> [1]
(and so on)
...
...
~~~

Note that rule confidence should be rounded to the nearest 2 decimal places (use `round(conf, 2)`). This table should also be tab (`'\t'`) separated. The rules should be displayed in the sorted order. If a rule is given as `LHS -> RHS` then the rules for which `len(LHS)` is lesser should appear first. If the `len(LHS)` is equal for two rules then rules for which `len(RHS)` is lesser should appear first. If both `len(LHS)` and `len(RHS)` is equal then the rules should be sorted based on LHS first and then based on RHS.

~~~~
Note:
LHS (Left Hand Side)
RHS (Right Hand Side)
~~~~

For eg.
~~~~
[3] -> [2] should come before [1, 3] -> [4]
[4] -> [2] should come before [2] -> [3, 4]
[1, 3] -> [2] should come before [1, 5] -> [2]
[1, 2] -> [3] should come before [1, 2] -> [5]
~~~~

Note that **this order is very important** because your output will be checked using the `diff` command. The sample output for the `small_retail.txt` dataset with `min_conf = 0.5` is:

~~~~
Sup	 Conf	Rule
0.44	0.67	[1] -> [2]
0.44	0.67	[1] -> [3]
0.44	0.57	[2] -> [1]
0.44	0.57	[2] -> [3]
0.44	0.67	[3] -> [1]
0.44	0.67	[3] -> [2]
0.22	1.0	 [4] -> [2]
0.22	1.0	 [5] -> [1]
0.22	1.0	 [5] -> [2]
0.22	1.0	 [5] -> [1, 2]
0.22	0.5	 [1, 2] -> [3]
0.22	0.5	 [1, 2] -> [5]
0.22	0.5	 [1, 3] -> [2]
0.22	1.0	 [1, 5] -> [2]
0.22	0.5	 [2, 3] -> [1]
0.22	1.0	 [2, 5] -> [1]
~~~~

**Store** this output for the `large_retail.txt` dataset in the file `apriori_rules.txt`. The sample output file for the `small_retail.txt` dataset has been provided to you as `small_apriori_rules.txt` for your convenience.

**Hint:** You shouldn't traverse the entire dataset to compute the confidence for a rule since you have already computed the `support_data` for all the frequent itemsets. 

In [9]:
class Rule:
    
    def __init__(self, sup, conf, fst, snd):
        self.support = sup
        self.confidence = conf
        self.first = fst
        self.second = snd

In [10]:
# your code goes here
large_dataset = load_dataset('large_retail.txt')
results = apriori_freq_temsets(large_dataset, 300)
retlst, support_data = filter_candidates(results, large_dataset, 300)
min_conf = 0.5
rules = []

f = open("apriori_rules.txt","w+")

for itemset, sup in support_data.items():
    if len(list(itemset)) > 1:
        for x in itemset:
            conf = support_data[frozenset(itemset)]/support_data[frozenset(itemset).difference(frozenset([x]))]
            if conf >= min_conf:
                support = sup/len(large_dataset)
                rules.append(Rule(support,conf,frozenset(itemset).difference(frozenset([x])),[x]))

rules.sort(key=lambda t: (len(t.first), list(t.first), list(t.second)))
print("Sup\tFreq Itemset\n")
f.write("Sup\tFreq Itemset\n")
count = 0
for rule in rules:
    print(str(round(rule.support,2)) + '\t' + str(round(rule.confidence,2)) + '\t' + str(list(rule.first)) + ' -> '+ str(list(rule.second)))
    if count < len(rules)-1:
        f.write(str(round(rule.support,2)) + '\t' + str(round(rule.confidence,2)) + '\t' + str(list(rule.first)) + ' -> '+ str(list(rule.second)) + '\n')
    else:
        f.write(str(round(rule.support,2)) + '\t' + str(round(rule.confidence,2)) + '\t' + str(list(rule.first)) + ' -> '+ str(list(rule.second)))
    count = count +1 
f.close()

Sup	Freq Itemset

0.14	0.65	[32] -> [39]
0.12	0.55	[32] -> [48]
0.17	0.67	[38] -> [39]
0.33	0.55	[39] -> [48]
0.23	0.74	[41] -> [39]
0.18	0.57	[41] -> [48]
0.33	0.7	[48] -> [39]
0.14	0.61	[41, 39] -> [48]
0.14	0.8	[48, 41] -> [39]


In [16]:
# your code goes here
large_dataset = load_dataset('small_retail.txt')
results = apriori_freq_temsets(large_dataset, 2)
retlst, support_data = filter_candidates(results, large_dataset, 2)
min_conf = 0.5
rules = []

f = open("s_apriori_rules.txt","w+")

for itemset, sup in support_data.items():
    if len(list(itemset)) > 1:
        for x in itemset:
            conf = support_data[frozenset(itemset)]/support_data[frozenset(itemset).difference(frozenset([x]))]
            if conf >= min_conf:
                support = sup/len(large_dataset)
                rules.append(Rule(support,conf,frozenset(itemset).difference(frozenset([x])),[x]))

rules.sort(key=lambda t: (len(t.first), list(t.first), list(t.second)))
print("Sup\tFreq Itemset\n")
f.write("Sup\Conf\tRule\n")
count = 0
for rule in rules:
    print(str(round(rule.support,2)) + '\t' + str(round(rule.confidence,2)) + '\t' + str(list(rule.first)) + ' -> '+ str(list(rule.second)))
    if count < len(rules)-1:
        f.write(str(round(rule.support,2)) + '\t' + str(round(rule.confidence,2)) + '\t' + str(list(rule.first)) + ' -> '+ str(list(rule.second)) + '\n')
    else:
        f.write(str(round(rule.support,2)) + '\t' + str(round(rule.confidence,2)) + '\t' + str(list(rule.first)) + ' -> '+ str(list(rule.second)))
    count = count +1 
f.close()

Sup	Freq Itemset

0.44	0.67	[1] -> [2]
0.44	0.67	[1] -> [3]
0.44	0.57	[2] -> [1]
0.44	0.57	[2] -> [3]
0.44	0.67	[3] -> [1]
0.44	0.67	[3] -> [2]
0.22	1.0	[4] -> [2]
0.22	1.0	[5] -> [1]
0.22	1.0	[5] -> [2]
0.22	0.5	[1, 2] -> [3]
0.22	0.5	[1, 2] -> [5]
0.22	0.5	[1, 3] -> [2]
0.22	1.0	[1, 5] -> [2]
0.22	0.5	[2, 3] -> [1]
0.22	1.0	[2, 5] -> [1]
